---

<h2 style="text-align: center;"><b>Word embeddings</b></h2>
<br> Присутствуют обучающие материалы из DLS

Библиотеки, которые нам понадобятся сегодня (на самом деле 2 первые из них вам тоже понадобятся впоследствии)

In [3]:
!pip install --upgrade nltk gensim bokeh

**NLTK** (Natural Language Toolkit) -- библиотека с множеством функций для работы с естественным языком (например, лемматизация, токенизация и т.д.)

https://www.nltk.org

We wull use it for tokenization for now.

In [4]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

quora = list(open("/content/drive/MyDrive/NLP/Lec1/quora.txt"))
# посмотрите на 10-й вопрос из набора данных
print(tokenizer.tokenize(quora[10]))

FileNotFoundError: ignored

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Давайте выделим все тексты из quora.txt файл и получаем массив массивов токенов:

In [4]:
quora_tokenized = [tokenizer.tokenize(line.lower()) for line in quora]

Существует множество различных моделей для обучения встраиванию слов (Word2Vec, GloVe, Fast Text и т.д.). А пока давайте попробуем обучить word2vec.

Библиотека gensim поможет нам в этом - она предоставляет модели для обучения Word2Vec и дальнейшего исследования векторов

In [6]:
from gensim.models import Word2Vec
# определите модель (точно так же, как модель torch) с параметрами
# и обучим его на наших данных
model = Word2Vec(quora_tokenized, # data for model to train on
                # size=32,         # embedding vector size
                 min_count=5,     # consider words that occured at least 5 times
                 window=3).wv     # define context as a 3-word window around the target word

Now we have trained model and can play with it:

In [7]:
# мы можем получить вектор для любого слова в нашем словаре
model.get_vector('tv')

array([ 2.05028   ,  1.1732328 ,  1.8690771 ,  3.115047  , -0.52948475,
       -0.37871805, -1.6909506 ,  0.95436454, -0.7006884 , -0.5957786 ,
       -0.59806633,  1.9189829 ,  1.3974497 , -0.74520665,  0.8385884 ,
        0.64575195,  0.35078266, -0.35600105,  0.13039918, -0.01832051,
        2.3652403 , -0.20920892,  0.20708306,  0.35353777,  0.7634945 ,
        0.58841604, -0.41782123,  0.45740426,  1.6559302 ,  1.2203902 ,
       -1.0471364 ,  0.1293066 ,  0.2071299 ,  1.3013158 ,  0.2741603 ,
       -0.32199878, -0.29142377, -0.19762284, -0.6343401 , -2.7536442 ,
        0.54261476, -3.2708266 , -0.3316612 ,  0.7168067 , -0.15625274,
       -0.64098346,  0.2939145 , -0.7505192 , -0.95889235,  0.00491759,
        2.219831  , -0.56373864, -1.5212404 ,  0.7117865 , -0.143657  ,
        0.5739159 ,  0.6202563 ,  0.72549   ,  1.1450355 ,  0.40056726,
        0.9561764 ,  2.8547065 , -1.3626566 ,  1.0202392 , -0.3783986 ,
       -0.6654603 ,  1.2546318 , -1.3208002 ,  1.3429047 ,  1.99

In [8]:
# и получите слова, векторы которых наиболее похожи на вектор данного слова
model.most_similar('tv')

[('television', 0.8076784610748291),
 ('netflix', 0.6535308361053467),
 ('hbo', 0.6359075307846069),
 ('anime', 0.6158614754676819),
 ('film', 0.6150400638580322),
 ('dvd', 0.5907697677612305),
 ('movies', 0.5892204642295837),
 ('comedy', 0.5844661593437195),
 ('lcd', 0.5835879445075989),
 ('kapil', 0.5830109715461731)]

Cуществуют предварительно обученные модели или даже предварительно вычисленные коллекции векторов слов!

In [9]:
import gensim.downloader as api
model = api.load('glove-twitter-100') # list of available models: https://github.com/RaRe-Technologies/gensim-data#models

[=================================================-] 99.7% 385.9/387.1MB downloaded


In [12]:
# who is a person who has money and related to coding, but is really stupid?
# (it's how a model trained on text corpus thinks, not me)
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

Также, используя gensim, мы можем загрузить предварительно обученные векторы из файла vectors. Например, мы можем загрузить предварительно подготовленные векторы Google word2vec::

http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/

In [ ]:
# from gensim.models import KeyedVectors
# model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

## Visualization

Итак, давайте визуализируем наши вложения в двумерном пространстве.

Что нам нужно, так это:
1. получите вложения для пакета слов из модели gensim
2. уменьшите размерность этих вложений со 100-мерных до 2-мерных векторов
3. нормализуйте эти векторы
4. нарисуйте это на плоскости!

Так что давайте сделаем это:

In [27]:
n_words = 1000
words = sorted(model.key_to_index.keys(),
               key=lambda word: model.get_vecattr(word, "count"), # sort by number of word occurencies
               reverse=True)[:n_words]

print(words[::10])

['<user>', 'i', ')', 'no', 'la', 'o', '>', 'are', 'we', '♥', '_', 'now', '~', 'people', "'re", '>>', '[', 'q', 'je', 'never', 'please', 'c', 'hahaha', 'لا', 'then', 'itu', 'had', 'ah', 'esta', 'does', 'apa', 'les', 'work', 'god', 'ha', 'their', 'в', '\\', 'gue', '＾', 'justin', 'thing', 'hope', 'nigga', 'udah', 'same', 'voy', 'find', 'اللي', 'face', 'text', 'kau', 'very', 'may', 'pretty', 'already', 'qué', 'día', 'money', 'name', 'hari', 'wow', 'black', 'vamos', 'nao', 'head', 'trying', 'lot', 'ان', 'mad', 'playing', '✌', 'own', 'friday', 'estar', 'yet', 'une', 'mana', 'لي', 'car', 'once', 'nem', 'summer', 'fan', 'tudo', 'sexy', 'sea', 'excited', 'far', 'lost', 'sei', 'sabe', 'picture', 'dice', 'km', 'fast', 'moi', 'anda', 'noche', 'relationship']


In [28]:
# get embeddings for those words
word_vectors = [model.get_vector(word) for word in words]

For vectors visualization we need to reduce dimensionality of vectors to 2 (or 3). We'll use **PCA** for that

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [29]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(word_vectors)
word_vectors_pca = pca.transform(word_vectors)

Also let's normalize vectors we got from PCA for better visibility

We already did normalization by hand in one of the first homeworks in our course, but for now let's use sklearn api:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [30]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(word_vectors_pca)
word_vectors_pca = ss.transform(word_vectors_pca)

Finally we are to draw an (interactive!) space of embeddings, using function below:

In [31]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

figure(id='p1004', ...)

Well, as we can see, this space does not look perfect, sometimes we see (it seems) different words grouped together.

That happend mostly because our way to reduce dimensionality of vectors (PCA)  is not perfect. Let's try to use different algorithm for doing this: TSNE.

TSNE is in some way creates embeddings itself, so we may hope that it will work better)

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

https://distill.pub/2016/misread-tsne/

In [1]:
from sklearn.manifold import TSNE
import numpy as np

word_vectors = [model[word] for word in words]
word_vectors = np.array(word_vectors)

tsne = TSNE(n_components=2, verbose=50)
word_vectors_tsne = tsne.fit_transform(word_vectors)

ss = StandardScaler().fit(word_vectors_tsne)
word_vectors_tsne = ss.transform(word_vectors_tsne)

print(words[::10])

NameError: ignored

In [ ]:
output_notebook()
draw_vectors(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], token=words)